<a href="https://colab.research.google.com/github/eringgao/lucy_listings/blob/main/lucyslistings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and Declarations

In [52]:
import pandas as pd
import numpy as np
from geopy import distance
import requests
import json
import datetime
import math
import itertools
import json
from abc import ABC, abstractmethod

# Data Processsing

In [54]:
df = pd.read_csv("Data/rapid_data.csv")
cmu_df = pd.read_csv("Data/CMU_listing_data.csv")

df = df.drop(columns = ["currency", "daysOnZillow", "homeStatusForHDP", "zpid", "datePriceChanged", "priceChange", "priceReduction", "taxAssessedValue", "isFeatured", "isNonOwnerOccupied", "priceForHDP", "shouldHighlight", "homeStatus", "isPreforeclosureAuction", "isPremierBuilder", "isZillowOwned", "isRentalWithBasePrice", "isUnmappable"])

In [57]:
addresses = df.loc[:, "streetAddress"]
bath = df.loc[:, "bathrooms"]
beds = df.loc[:, "bedrooms"]
city = df.loc[:, "city"]
state = df.loc[:, "state"]
house_type = df.loc[:, "homeType"]
lat = df.loc[:, "latitude"]
longi = df.loc[:, "longitude"]
price = df.loc[:, "price"]
zip = df.loc[:, "zipcode"]
cmu_address = (40.4433, -79.944023)
distance_df = pd.DataFrame(columns = ['addr', 'Latitude', 'Longitude'])

distance_df['addr'] = addresses
distance_df['bath'] = bath
distance_df['beds'] = beds
distance_df['city'] = city
distance_df['state'] = state
distance_df['house_type'] = house_type
distance_df['Latitude'] = lat
distance_df['Longitude'] = longi
distance_df['price'] = price
distance_df['zip'] = zip

In [58]:
walking = []
driving = []
for i in range(len(addresses)):
  d = distance.distance((distance_df.loc[i, "Latitude"], distance_df.loc[i, "Longitude"]), (cmu_address[0],cmu_address[1]))
  walking.append(d.miles)
  r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{distance_df.loc[i, "Longitude"]},{distance_df.loc[i, "Latitude"]};{cmu_address[1]},{cmu_address[0]}?overview=false""")
  route_1 = json.loads(r.content)["routes"][0]
  driving.append(route_1["distance"]*0.000621371)
distance_df["walk"] = walking
distance_df["drive"] = driving

In [60]:
distance_df = pd.read_csv('data/distance_df.csv')
print(distance_df)

    Unnamed: 0                           addr   Latitude  Longitude  bath  \
0            0              2353 Sidney St #2  40.429380 -79.971140   2.0   
1            1             441 Edgemont St #2  40.422600 -79.998880   1.0   
2            2                  200 S 15th St  40.424866 -79.983440   1.5   
3            3                5317 Forbes Ave  40.440155 -79.934940   2.0   
4            4             4711 Maripoe St #1  40.455498 -79.949670   1.0   
5            5                  1625 Hybla St  40.474262 -80.037970   1.0   
6            6            3202 Hazelhurst Ave  40.383247 -79.965300   1.0   
7            7               4245 Parkman Ave  40.448345 -79.957280   3.0   
8            8             1007 Galveston Ave  40.453010 -80.017100   1.5   
9            9                11370 Althea Rd  40.468670 -79.832700   2.0   
10          10              333 Lehigh Ave #2  40.456150 -79.926100   2.0   
11          11  1465 Old Steubenville Pike #B  40.450570 -80.114400   1.0   

# OOP Structuring

In [40]:
distance_df = pd.read_csv('data/distance_df.csv')

In [41]:
class Listing(ABC): # abstraction?
  def __init__(self, addr_dict, price, walk_time=None, drive_dist=None, h_type=None, description=None): 
    self.address = addr_dict
    self.house_type = h_type 
    self.walk = walk_time
    self.drive = drive_dist
    self.price = price
    self.description = description

  def get_housing_info(self): 
    return self.housing_info

  def update_housing_info(self, new_info): 
    # pass
    self.housing_info = new_info

  def get_address(self): 
    return self.address

  def get_drive_distance(self): 
    return self.drive

  @abstractmethod
  def get_list_type(self): 
    pass


class ZillowListing(Listing): 
  def __init__(self, addr_dict, price, walk_time=None, drive_dist=None, h_type=None, description=None, zlink=None): 
    super().__init__(addr_dict, price, walk_time=None, drive_dist=None, h_type=None, description=None)
    self.zillow_link = zlink

  def get_list_type(self): 
    return "Zillow"
  
  def get_zillow_link(self):
    return self.zillow_link

  def update_zillow_link(self, new_link):
    self.zillow_link = new_link


  


In [42]:
zill_listings = [ZillowListing({'addr':row.addr, 'city':row.city, 'state':row.state, 'zipcode':row.zip}, row.price, row.walk, row.drive, row.house_type) for _, row in distance_df.iterrows()]

In [43]:

class CMUListing(Listing):
    def __init__(self, addr_dict, grade, building_name, room_type, bathroom_type, price, ac):
        super().__init__(addr_dict, price)
        self.building_name = building_name
        self.grade = grade
        self.room_type = room_type
        self.bathroom_type = bathroom_type
        self.price = price
        self.ac = ac   
    def get_list_type(self):
        return "CMU On-Campus"
    def get_grade(self):
        return self.grade
    
class ManualListing(Listing):
    def __init__(self, first_name, last_name, contact, availability, addr_dict, bathrooms, bedrooms, home_type):
        super().__init__(addr_dict, price)
        self.first_name = first_name
        self.last_name = last_name
        self.contact = contact
        self.availability = availability 
        self.bathrooms = bathrooms
        self.bedrooms = bedrooms
        self.home_type = home_type
    def get_list_type(self):
        return "User Posting"

In [44]:
df_cmu = pd.read_csv("Data/CMU_listing_data.csv")
address = []
cities = []
states = []
zipcodes = []
for i in df_cmu.loc[:, "Address"]:
    if(i == "Corner of Morewood Avenue and Forbes Avenue"):
        i = "1065 Morewood Ave, Pittsburgh, PA 15213"
    if(i == "Margaret Morrison Corridor"):
        i = "5143 Forbes Ave, Pittsburgh, PA 15213"
    splits = i.split(',')
    address.append(splits[0])
    cities.append(splits[1])
    states.append(splits[2].split(" ")[0])
    zipcodes.append(splits[2].split(" ")[1])
cmu_listings = []
i = 0
for _, row in df_cmu.iterrows():
    cmu_listings.append(CMUListing({'addr':address[i], 'city':cities[i], 'state':states[i], 'zipcode':zipcodes[i]}, row.For, row.Building, row.Room, row.Bathroom, row.Cost, row.AC))
    i+=1
for listing in cmu_listings: 
  print(listing.get_grade())

Freshman
Freshman
Freshman
Freshman
Freshman
Freshman
Freshman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Freshman
Freshman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Freshman
Freshman
Freshman
Freshman
Freshman
All
All
All
Freshman
Freshman
Freshman
Freshman
Freshman
Freshman
Upperclassman
Upperclassman
Freshman
Freshman
Freshman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Freshman
Freshman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Freshman
Freshman
Freshman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman
Upperclassman


In [49]:
cmu_jstr = json.dumps([ob.__dict__ for ob in cmu_listings], indent=4)
zil_jstr = json.dumps([ob.__dict__ for ob in zill_listings], indent=4)
# f = open('/Users/margarethe/lucy_listings/data/cmu_list.json', 'w')
# f.write(cmu_jstr)
# f.close()